In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import tree
import xgboost as xgb
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [2]:
train = pd.read_csv('C:/Users/sangkyu/git/kaggle/titanic/input/train.csv').set_index('PassengerId')
test = pd.read_csv('C:/Users/sangkyu/git/kaggle/titanic/input/test.csv').set_index('PassengerId')

In [3]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# train,test 모두 전처리를 해주어야 하기 때문에 묶었다.
df = pd.concat([train, test], axis=0, sort=False)

In [6]:
# 여자와 Title이 Master인 소년들을 새로운 Column으로 만들었음
df['Title'] = df.Name.str.split(',').str[1].str.split('.').str[0].str.strip()
df['IsWomanOrChild'] = ((df.Title == 'Master') | (df.Sex == 'female'))
df['LastName'] = df.Name.str.split(',').str[0]
family = df.groupby(df['LastName']).Survived
df['FamilyTotalCount'] = family.transform(lambda s: s[df.IsWomanOrChild].fillna(0).count())
df['FamilyTotalCount'] = df.mask(df.IsWomanOrChild, df.FamilyTotalCount - 1, axis=0)
df['FamilySurvivedCount'] = family.transform(lambda s: s[df.IsWomanOrChild].fillna(0).sum())
df['FamilySurvivedCount'] = df.mask(df.IsWomanOrChild, df.FamilySurvivedCount - df.Survived.fillna(0), axis=0)
df['FamilySurvivalRate'] = (df.FamilySurvivedCount / df.FamilyTotalCount.replace(0, np.nan))
df['IsSingleTraveler'] = df.FamilyTotalCount == 0
df['FamilySurvivalRate']=df.FamilySurvivalRate.fillna(0)
df['Sex']=df.Sex.replace({'male':0,'female':1})

In [23]:
x = pd.concat([
    df.FamilySurvivalRate,
    df.IsSingleTraveler,
    df.Sex,
    df.Survived
], axis=1)
x.head()

,FamilySurvivalRate,IsSingleTraveler,Sex,Survived
PassengerId,,,,
1,0.0,True,0,0.0
2,0.0,True,1,1.0
3,0.0,True,1,1.0
4,0.0,True,1,1.0
5,0.0,True,0,0.0


In [19]:
x = pd.concat([
    df.FamilySurvivalRate,
    df.IsSingleTraveler,
    df.Sex,
    df.Survived
], axis=1)
train ,test = x.loc[train.index], x.loc[test.index]
X=train.drop(columns=['Survived'],axis=1)
y=train['Survived']
test_f = test.drop(columns=['Survived'],axis=1)
# y = df.Survived.loc[train.index]

In [20]:
x

,FamilySurvivalRate,IsSingleTraveler,Sex,Survived
PassengerId,,,,
1,0.0,True,0,0.0
2,0.0,True,1,1.0
3,0.0,True,1,1.0
4,0.0,True,1,1.0
5,0.0,True,0,0.0
6,0.0,True,0,0.0
7,0.0,True,0,0.0
8,0.0,False,0,0.0
9,1.0,False,1,1.0


In [ ]:
test.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
model1 = DecisionTreeClassifier()
model1.fit(x_train, y_train)
val_pred1=pd.DataFrame(model1.predict(x_val))
test_pred1=pd.DataFrame(model1.predict(x_test))

model2 = KNeighborsClassifier()
model2.fit(x_train,y_train)
val_pred2=pd.DataFrame(model2.predict(x_val))
test_pred2=pd.DataFrame(model2.predict(x_test))

model3 = RandomForestClassifier()
model3.fit(x_train,y_train)
val_pred3=pd.DataFrame(model3.predict(x_val))
test_pred3=pd.DataFrame(model3.predict(x_test))

model4 = GradientBoostingClassifier()
model4.fit(x_train,y_train)
val_pred4=pd.DataFrame(model4.predict(x_val))
test_pred4=pd.DataFrame(model4.predict(x_test))

In [ ]:
x_val.head()

In [ ]:
df_val=pd.concat([x_val.reset_index(drop=True), val_pred1.reset_index(drop=True),val_pred2.reset_index(drop=True),val_pred3.reset_index(drop=True),val_pred4.reset_index(drop=True)],axis=1)
df_test=pd.concat([x_test.reset_index(drop=True), test_pred1.reset_index(drop=True),test_pred2.reset_index(drop=True),test_pred3.reset_index(drop=True),test_pred4.reset_index(drop=True)],axis=1)

In [ ]:
df_val.head()

In [ ]:
df_test.head()

In [ ]:
model = LogisticRegression()
model.fit(df_val,y_val)
model.score(df_test,y_test)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
model1 = DecisionTreeClassifier()
model1.fit(x_train, y_train)
val_pred1=pd.DataFrame(model1.predict(x_val))
test_pred1=pd.DataFrame(model1.predict(test_f))

model2 = KNeighborsClassifier()
model2.fit(x_train,y_train)
val_pred2=pd.DataFrame(model2.predict(x_val))
test_pred2=pd.DataFrame(model2.predict(test_f))

model3 = RandomForestClassifier()
model3.fit(x_train,y_train)
val_pred3=pd.DataFrame(model3.predict(x_val))
test_pred3=pd.DataFrame(model3.predict(test_f))

model4 = GradientBoostingClassifier()
model4.fit(x_train,y_train)
val_pred4=pd.DataFrame(model4.predict(x_val))
test_pred4=pd.DataFrame(model4.predict(test_f))

In [ ]:
x_val.head()

In [ ]:
df_val=pd.concat([x_val.reset_index(drop=True), val_pred1.reset_index(drop=True),val_pred2.reset_index(drop=True),val_pred3.reset_index(drop=True),val_pred4.reset_index(drop=True)],axis=1)
df_test=pd.concat([test_f.reset_index(drop=True), test_pred1.reset_index(drop=True),test_pred2.reset_index(drop=True),test_pred3.reset_index(drop=True),test_pred4.reset_index(drop=True)],axis=1)

In [ ]:
df_val.tail()

In [ ]:
model = LogisticRegression()
model.fit(df_val,y_val)
y_target = model.predict(df_test)

In [ ]:
test_salida = pd.DataFrame( { 'PassengerId': test_f.index , 'Survived': y_target } ).astype(int)

In [ ]:
test_salida.head(20)

In [ ]:
test_salida['Survived'].value_counts()

In [ ]:
test_salida.to_csv( 'titanic_pred.csv' , index = False )

In [ ]:
blend = pd.read_csv('C:/Users/sangkyu/git/kaggle/titanic/submission/blended.csv')
blend.head(5)

In [ ]:
for i in range(0, len(blend)):
    for c in range(3, 10):
        blend.loc[i, 'sum'] = blend.loc[i, 'sum'] + blend.iloc[i,c]
        
    if blend.loc[i, 'sum'] > 2.5:
        blend.loc[i, 'Survived'] = 1
            
    if blend.loc[i, 'sum'] < 2.5:
        blend.loc[i, 'Survived'] = 0

blend.head(10)  

In [ ]:
blend = blend.iloc[:, 0:2]
blend.to_csv('kernel_blend_1st.csv', index=False)

In [ ]:
blend2 = pd.read_csv('C:/Users/sangkyu/git/kaggle/titanic/submission/blended2.csv')
blend2.head(5)